### 1. Carga de datos (ingesta analítica)

Carga `Master_Features.csv` y muestra un estado rápido. La celda siguiente lee el CSV.

In [ ]:
print("--- [1/4] Cargando Master Features ---")
try:
    df = pd.read_csv('Master_Features.csv')
    print(f"Status: OK. {len(df)} clientes cargados.")
except FileNotFoundError:
    print("Error: No se encuentra 'Master_Features.csv'. Ejecuta el Paso 2 primero.")


### 2. Machine Learning — Scoring de propensión

Entrena un modelo sencillo (Random Forest) y calcula el `Propensity_Score` para cada cliente.

La siguiente celda entrena y aplica el modelo.

In [ ]:
print("\n--- [2/4] Entrenando Modelo de Propensión (Random Forest) ---")

# A. Definir el Target (Variable Objetivo)
# En un proyecto real, esto sería "Compró en el mes siguiente".
# Aquí simulamos un target lógico: Reciente (<90 días) Y Frecuente (>=1 compra)
df['Target_Will_Buy'] = np.where(
    (df['recency_days'] < 90) & (df['frequency'] >= 1), 
    1, 0
)

# B. Selección de Features (Variables Predictoras)
# Solo usamos datos numéricos de comportamiento
features = ['recency_days', 'frequency', 'monetary', 'avg_ticket', 'total_page_views', 'total_add_to_cart']
X = df[features]
y = df['Target_Will_Buy']

# C. Entrenamiento
# Usamos Random Forest porque es robusto, no requiere escalado de datos 
# y es interpretable (feature importance).
# n_estimators=100: Crea 100 árboles de decisión para votar.
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
model.fit(X, y)

# D. Scoring
# Predecimos la probabilidad para TODOS los clientes (incluso los que no compraron)
df['Propensity_Score'] = model.predict_proba(X)[:, 1]

print("Status: Modelo entrenado.")
print(f"Ejemplo de Score: Cliente {df.iloc[0]['user_id']} -> Probabilidad: {df.iloc[0]['Propensity_Score']:.2f}")


### 3. Reglas de negocio — Motor de decisiones

Convertir el `Propensity_Score` en una acción concreta usando reglas simples.

La celda siguiente aplica esas reglas por cliente.

In [ ]:
print("\n--- [3/4] Aplicando Motor de Reglas de Negocio ---")

def get_next_best_action(row):
    """
    Función que decide la estrategia para cada cliente.
    Prioridad: Rentabilidad > Urgencia > Retención > Nurturing.
    """
    score = row['Propensity_Score']
    recency = row['recency_days']
    abandoned = row['is_cart_abandoner']
    monetary = row['monetary']
    
    # --- REGLA 1: FILTRO DE RENTABILIDAD (Anti-Loss) ---
    # Si el cliente devuelve más de lo que compra (valor negativo), no gastar presupuesto.
    if monetary < 0:
        return "DO_NOTHING_NEGATIVE_VALUE"

    # --- REGLA 2: URGENCIA (Conversion) ---
    # Si tiene alta intención de compra (>60%) Y abandonó el carrito -> SMS (Canal caro pero efectivo)
    if score > 0.60 and abandoned == 1:
        return "SMS_RECOVER_CART_20_OFF"
    
    # --- REGLA 3: PROTECCIÓN DE MARGEN (Smart Yield) ---
    # Si la probabilidad es casi segura (>95%) -> NO enviar descuento.
    # Enviar novedades para maximizar el ticket.
    if score > 0.95:
        return "PUSH_NEW_COLLECTION_NO_PROMO"
        
    # --- REGLA 4: RETENCIÓN (Winback) ---
    # Buenos clientes históricos (Frecuencia > 2) que se están enfriando (Recencia > 100 días).
    # Necesitan "cariño" (Storytelling), no venta dura.
    if row['frequency'] > 2 and recency > 100:
        return "EMAIL_WE_MISS_YOU_STORYTELLING"
        
    # --- REGLA 5: NURTURING (Mantenimiento) ---
    # Clientes con score decente pero sin urgencia -> Newsletter semanal estándar.
    if score > 0.30:
        return "EMAIL_WEEKLY_NEWSLETTER"
        
    # Default: Clientes fríos o de bajo valor
    return "DO_NOTHING_LOW_SCORE"

# Aplicar lógica fila a fila
df['Next_Best_Action'] = df.apply(get_next_best_action, axis=1)



--- [3/4] Aplicando Motor de Reglas de Negocio ---


### 4. Exportación y resultados

Guardar la lista final con `Next_Best_Action` y mostrar un resumen rápido.

In [ ]:
print("\n--- [4/4] Resultados Finales ---")

# Resumen Ejecutivo
print("Distribución de Acciones Sugeridas:")
summary = df['Next_Best_Action'].value_counts()
print(summary)

# Guardar CSV final para integración (Paso 4)
output_cols = ['user_id', 'email_clean', 'Propensity_Score', 'Next_Best_Action']
df[output_cols].to_csv('Final_Action_List.csv', index=False)

print("\nArchivo 'Final_Action_List.csv' generado correctamente.")
print("Listo para Reverse ETL.")



--- [4/4] Resultados Finales ---
Distribución de Acciones Sugeridas:
Next_Best_Action
DO_NOTHING_LOW_SCORE              193
EMAIL_WE_MISS_YOU_STORYTELLING    150
SMS_RECOVER_CART_20_OFF           112
EMAIL_WEEKLY_NEWSLETTER            23
DO_NOTHING_NEGATIVE_VALUE          18
PUSH_NEW_COLLECTION_NO_PROMO       14
Name: count, dtype: int64

Archivo 'Final_Action_List.csv' generado correctamente.
Listo para Reverse ETL.
